## Feature Engineering & Preprocessing (Preparing ML-Ready Data)

Feature Engineering is the process of creating, selecting, and transforming variables (features) to improve the performance of machine learning models.

In this notebook, we will apply several techniques to transform the cleaned dataset into a more informative and ML-ready format.

### Objectives of this step:

1. **Remove or encode non-numeric columns**  
   Convert categorical variables to numeric formats using techniques such as One-Hot Encoding or Label Encoding.

2. **Handle date or time-related features**  
   Derive new features from construction year (e.g., building age), or other temporal indicators.

3. **Create new derived features**  
   Add variables like:
   - `building_age` = `current_year - buildingConstructionYear`

4. **(Optional) Normalize or scale selected features**  
   This step is not required for tree-based models (e.g., XGBoost, Random Forest), but might be included later if experimenting with distance-based algorithms.

5. **Reduce dimensionality or drop irrelevant features**  
   Focus only on the most relevant variables based on domain knowledge or correlation analysis.

At the end of this notebook, we will generate a dataset ready for training baseline models.


In [21]:
import sys, os

# Add the project root to the Python path
project_root = os.path.abspath("../..")
sys.path.append(project_root)

# Imports from local modules
import pandas as pd
from utils.data_cleaner import DataCleaner
from utils.data_loader import DataLoader
from utils.constants import CLEANED_DATA_FILE, LEAK_FEATURES


#Load and clean the dataset

loader = DataLoader(CLEANED_DATA_FILE)
df = loader.load_data()

df = loader.clean_booleans(df, bool_cols=["hasLivingRoom", "hasTerrace"])

df = loader.drop_columns(df, columns_to_drop=["Unnamed: 0", "id", "url"])
df = loader.drop_na_targets(df, target_col="price")

df.head()


,type,subtype,bedroomCount,bathroomCount,province,locality,postCode,habitableSurface,buildingCondition,buildingConstructionYear,facedeCount,floodZoneType,heatingType,kitchenType,hasLivingRoom,toiletCount,hasTerrace,epcScore,price
0,APARTMENT,APARTMENT,2.0,1.0,Brussels,Etterbeek,1040,100.0,GOOD,2004.0,1.0,NON_FLOOD_ZONE,GAS,SEMI_EQUIPPED,1,1.0,1,C,399000.0
1,HOUSE,HOUSE,4.0,2.0,Brussels,Etterbeek,1040,270.0,TO_BE_DONE_UP,1910.0,2.0,NON_FLOOD_ZONE,FUELOIL,INSTALLED,1,3.0,1,E,895000.0
2,APARTMENT,APARTMENT,2.0,1.0,Brussels,Etterbeek,1040,87.0,AS_NEW,1970.0,2.0,NaN,FUELOIL,HYPER_EQUIPPED,1,1.0,1,F,465000.0
3,APARTMENT,APARTMENT,2.0,2.0,Brussels,Bruxelles,1040,104.0,AS_NEW,2018.0,NaN,RECOGNIZED_FLOOD_ZONE,GAS,HYPER_EQUIPPED,1,2.0,1,B,590000.0
4,APARTMENT,FLAT_STUDIO,1.0,1.0,Brussels,Etterbeek,1040,71.0,AS_NEW,1906.0,2.0,NON_FLOOD_ZONE,GAS,INSTALLED,0,1.0,0,E,289000.0


In [23]:
#  Create new derived features BEFORE pipeline
df["building_age"] = 2025 - df["buildingConstructionYear"]

In [ ]:
from utils.preprocessing_pipeline import PreprocessingPipeline


# 2. Initialize preprocessing pipeline

pipeline = PreprocessingPipeline(
    df=df,
    target_col="price",
    drop_cols=["price_per_m2"],  
)

df_encoded = pipeline.fit_transform()


# 3. Save full and sample dataset

import os
import shutil
from utils.constants import ML_READY_DIR, ML_READY_DATA_FILE, ML_READY_SAMPLE_XLSX

# Clean and recreate ml_ready directory
if os.path.exists(ML_READY_DIR):
    shutil.rmtree(ML_READY_DIR)
os.makedirs(ML_READY_DIR, exist_ok=True)

# Save to CSV and Excel
df_encoded.to_csv(ML_READY_DATA_FILE, index=False)
df_encoded.head(10).to_excel(ML_READY_SAMPLE_XLSX, index=False)

print(f"Dataset ready. Shape: {df_encoded.shape}")
print(f"Saved to: {ML_READY_DATA_FILE}")
print(f"Excel sample: {ML_READY_SAMPLE_XLSX}")

Dataset ready. Shape: (30189, 3259)
Saved to: e:\_SoftEng\_BeCode\real-estate-price-predictor\data\ml_ready\immoweb_real_estate_ml_ready.csv
Excel sample: e:\_SoftEng\_BeCode\real-estate-price-predictor\data\ml_ready\immoweb_real_estate_ml_ready_sample10.xlsx
